In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import fisher_exact

In [2]:
def prueba_fisher_exact(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba chi cuadrada de independencia entre dos variables categóricas.
    Retorna un diccionario con resultados de la prueba.
    """

    # Crear tabla de contingencia
    tabla_contingencia = pd.crosstab(df[var1], df[var2])

    # Realizar prueba chi cuadrada
    chi2, p_valor  = fisher_exact(tabla_contingencia)

    # Determinar resultado
    resultado = ('Las variables son dependientes (Se rechaza la hipótesis nula)'
        if p_valor < alfa
        else 'Las variables son independientes (No se rechaza la hipótesis nula)')

    # Imprimir resultados
    print('Tabla de contingencia')
    print(tabla_contingencia)
    print('\nEstadístico de prueba: ', chi2.round(4))
    print('Valor p: ', p_valor.round(4))
    print('Resultado: ', resultado)

    # Retornar p-valor
    return p_valor

In [3]:
def prueba_chi_cuadrada(df, var1, var2, alfa=0.05, imprimir=True):
    """
    Realiza una prueba chi cuadrada de independencia entre dos variables categóricas.
    Retorna un diccionario con resultados de la prueba.
    """

    # Crear tabla de contingencia
    tabla_contingencia = pd.crosstab(df[var1], df[var2])

    # Realizar prueba chi cuadrada
    chi2, p_valor, grados_libertad, frecuencias_esperadas = chi2_contingency(tabla_contingencia)

    # Calcular proporción de celdas con frecuencia esperada menor a 5
    celdas_menor_5 = np.sum(frecuencias_esperadas < 5)
    total_celdas = frecuencias_esperadas.size
    prop_freq_menor_5 = celdas_menor_5 / total_celdas

    # Preparar advertencia sobre frecuencias esperadas si es necesario
    advertencia = None
    if prop_freq_menor_5 > 0.2:
        advertencia = ("ADVERTENCIA: Más del 20% de las celdas tienen frecuencias esperadas menores a 5. "
                       "Los resultados de la prueba podrían no ser confiables.")

    # Determinar resultado
    resultado = ('Las variables son dependientes (Se rechaza la hipótesis nula)'
        if p_valor < alfa
        else 'Las variables son independientes (No se rechaza la hipótesis nula)')

    # Imprimir resultados
    print('Tabla de contingencia')
    print(tabla_contingencia)
    print('\nEstadístico de prueba: ', chi2.round(4))
    print('Valor p: ', p_valor.round(4))
    print('Grados de libertad: ', grados_libertad)
    print('\nTabla de frecuencias esperadas:')
    print(pd.DataFrame(
        frecuencias_esperadas,
        index=tabla_contingencia.index,
        columns=tabla_contingencia.columns).round(4))
    print('\nProporción de celdas con frecuencia esperada menor a 5: ', prop_freq_menor_5.round(2))
    print(f'({celdas_menor_5} de {total_celdas} celdas)')
    if advertencia:
        print(f'\n{advertencia}')
    print('\nNivel de significancia: ', alfa)
    print('Resultado: ', resultado)

    # Retornar p-valor
    return p_valor

In [4]:
df = pd.read_excel(r"C:\Users\rodmo\Desktop\MBD Maestría EGADE\Twitter Scraping\Base_GOLD_Temas_Sentimientos.xlsx", sheet_name="BD_Gold")
df.head()

,ID_publicacion,ID_comentarios_list,sentimiento_comentario,tematica_analizada,Publicacion,Comentario
0,P-01,"Ch-01,Ch-02,Ch-03,Ch-04",Negativo,Politica - Liderazgo,sheinbaum en el camino de chavez y castroclaud...,eso en estados unidos la segunda enmienda es f...
1,P-010,"Ch-043,Ch-044,Ch-045,Ch-046",Neutro,Educacion,el carton del maestro calderon y la carta que ...,puedes amargar lo amargo
2,P-0100,"Ch-0374,Ch-0375,Ch-0376,Ch-0377",Negativo,Politica - Gobierno,falta de empatia es alarmante sheinbaum firmo ...,d angelo 4t una pedante pasmosidad este wey qu...
3,P-0101,"Ch-0378,Ch-0379,Ch-0380,Ch-0381,Ch-0382",Neutro,Seguridad,posible detencion del heredero del cartel de s...,iio culiacanazo
4,P-0102,"Ch-0383,Ch-0384,Ch-0385,Ch-0386",Negativo,Economia,sheinbaum se jubilara de la unam sin haber tra...,preocupada porque los empleados del poder judi...


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ID_publicacion            211 non-null    object
 1   ID_comentarios_list       205 non-null    object
 2   sentimiento_comentario    211 non-null    object
 3   tematica_analizada        211 non-null    object
 4   Publicacion               211 non-null    object
 5   Comentario                204 non-null    object
 6   tematica_analizada_2      211 non-null    object
 7   sentimiento_comentario_2  211 non-null    object
dtypes: object(8)
memory usage: 13.3+ KB


In [ ]:
df.query(not df['Comentario'].isblank() == True and )

In [5]:
tabla_contingencia = pd.crosstab(df['tematica_analizada'], df['sentimiento_comentario'])
tabla_contingencia

sentimiento_comentario,Negativo,Neutro,Positivo
tematica_analizada,,,
Economia,17,6,0
Educacion,5,4,1
Politica - Elecciones,11,4,0
Politica - Gobierno,15,6,2
Politica - Liderazgo,64,31,9
Politica - Partidos,5,2,0
Salud,4,3,0
Seguridad,8,13,1


In [6]:
df['tematica_analizada_2'] = df['tematica_analizada'].apply(lambda x: 'Politico' if 'Politica' in x else 'Social y Económico')
df.head()

,ID_publicacion,ID_comentarios_list,sentimiento_comentario,tematica_analizada,Publicacion,Comentario,tematica_analizada_2
0,P-01,"Ch-01,Ch-02,Ch-03,Ch-04",Negativo,Politica - Liderazgo,sheinbaum en el camino de chavez y castroclaud...,eso en estados unidos la segunda enmienda es f...,Politico
1,P-010,"Ch-043,Ch-044,Ch-045,Ch-046",Neutro,Educacion,el carton del maestro calderon y la carta que ...,puedes amargar lo amargo,Social y Económico
2,P-0100,"Ch-0374,Ch-0375,Ch-0376,Ch-0377",Negativo,Politica - Gobierno,falta de empatia es alarmante sheinbaum firmo ...,d angelo 4t una pedante pasmosidad este wey qu...,Politico
3,P-0101,"Ch-0378,Ch-0379,Ch-0380,Ch-0381,Ch-0382",Neutro,Seguridad,posible detencion del heredero del cartel de s...,iio culiacanazo,Social y Económico
4,P-0102,"Ch-0383,Ch-0384,Ch-0385,Ch-0386",Negativo,Economia,sheinbaum se jubilara de la unam sin haber tra...,preocupada porque los empleados del poder judi...,Social y Económico


In [7]:
tabla_contingencia = pd.crosstab(df['tematica_analizada_2'], df['sentimiento_comentario'])
tabla_contingencia

sentimiento_comentario,Negativo,Neutro,Positivo
tematica_analizada_2,,,
Politico,95,43,11
Social y Económico,34,26,2


In [8]:
prueba_fisher_exact(df, 'tematica_analizada_2', 'sentimiento_comentario', alfa=0.05, imprimir=True)

Tabla de contingencia
sentimiento_comentario  Negativo  Neutro  Positivo
tematica_analizada_2                              
Politico                      95      43        11
Social y Económico            34      26         2

Estadístico de prueba:  0.0048
Valor p:  0.1473
Resultado:  Las variables son independientes (No se rechaza la hipótesis nula)


np.float64(0.1473)

In [9]:
prueba_chi_cuadrada(df, 'tematica_analizada_2', 'sentimiento_comentario', alfa=0.05, imprimir=True)

Tabla de contingencia
sentimiento_comentario  Negativo  Neutro  Positivo
tematica_analizada_2                              
Politico                      95      43        11
Social y Económico            34      26         2

Estadístico de prueba:  4.0869
Valor p:  0.1296
Grados de libertad:  2

Tabla de frecuencias esperadas:
sentimiento_comentario  Negativo   Neutro  Positivo
tematica_analizada_2                               
Politico                 91.0948  48.7251    9.1801
Social y Económico       37.9052  20.2749    3.8199

Proporción de celdas con frecuencia esperada menor a 5:  0.17
(1 de 6 celdas)

Nivel de significancia:  0.05
Resultado:  Las variables son independientes (No se rechaza la hipótesis nula)


np.float64(0.12958003965723777)

In [ ]:
prueba_chi_cuadrada(df, 'tematica_analizada', 'sentimiento_comentario', alfa=0.05, imprimir=True)

Tabla de contingencia
sentimiento_comentario  Negativo  Neutro  Positivo
tematica_analizada                                
Economia                      17       6         0
Educacion                      5       4         1
Politica - Elecciones         11       4         0
Politica - Gobierno           15       6         2
Politica - Liderazgo          64      31         9
Politica - Partidos            5       2         0
Salud                          4       3         0
Seguridad                      8      13         1

Estadístico de prueba:  14.5742
Valor p:  0.4079
Grados de libertad:  14

Tabla de frecuencias esperadas:
sentimiento_comentario  Negativo   Neutro  Positivo
tematica_analizada                                 
Economia                 14.0616   7.5213    1.4171
Educacion                 6.1137   3.2701    0.6161
Politica - Elecciones     9.1706   4.9052    0.9242
Politica - Gobierno      14.0616   7.5213    1.4171
Politica - Liderazgo     63.5829  34.0095    6.40

np.float64(0.4078684727569657)

In [14]:
df['sentimiento_comentario_2'] = df['sentimiento_comentario'].apply(lambda x: 'Negativo' if 'Negativo' in x else 'Neutro o Positivo')
df.head()

,ID_publicacion,ID_comentarios_list,sentimiento_comentario,tematica_analizada,Publicacion,Comentario,tematica_analizada_2,sentimiento_comentario_2
0,P-01,"Ch-01,Ch-02,Ch-03,Ch-04",Negativo,Politica - Liderazgo,sheinbaum en el camino de chavez y castroclaud...,eso en estados unidos la segunda enmienda es f...,Politico,Negativo
1,P-010,"Ch-043,Ch-044,Ch-045,Ch-046",Neutro,Educacion,el carton del maestro calderon y la carta que ...,puedes amargar lo amargo,Social y Económico,Neutro o Positivo
2,P-0100,"Ch-0374,Ch-0375,Ch-0376,Ch-0377",Negativo,Politica - Gobierno,falta de empatia es alarmante sheinbaum firmo ...,d angelo 4t una pedante pasmosidad este wey qu...,Politico,Negativo
3,P-0101,"Ch-0378,Ch-0379,Ch-0380,Ch-0381,Ch-0382",Neutro,Seguridad,posible detencion del heredero del cartel de s...,iio culiacanazo,Social y Económico,Neutro o Positivo
4,P-0102,"Ch-0383,Ch-0384,Ch-0385,Ch-0386",Negativo,Economia,sheinbaum se jubilara de la unam sin haber tra...,preocupada porque los empleados del poder judi...,Social y Económico,Negativo


In [15]:
prueba_fisher_exact(df, 'tematica_analizada_2', 'sentimiento_comentario_2', alfa=0.05, imprimir=True)

Tabla de contingencia
sentimiento_comentario_2  Negativo  Neutro o Positivo
tematica_analizada_2                                 
Politico                        95                 54
Social y Económico              34                 28

Estadístico de prueba:  1.4488
Valor p:  0.2779
Resultado:  Las variables son independientes (No se rechaza la hipótesis nula)


np.float64(0.2778517454450412)

In [18]:
resultado = prueba_chi_cuadrada(df, 'tematica_analizada_2', 'sentimiento_comentario_2', alfa=0.05, imprimir=True)
print(resultado.round(2))

Tabla de contingencia
sentimiento_comentario_2  Negativo  Neutro o Positivo
tematica_analizada_2                                 
Politico                        95                 54
Social y Económico              34                 28

Estadístico de prueba:  1.1147
Valor p:  0.2911
Grados de libertad:  1

Tabla de frecuencias esperadas:
sentimiento_comentario_2  Negativo  Neutro o Positivo
tematica_analizada_2                                 
Politico                   91.0948            57.9052
Social y Económico         37.9052            24.0948

Proporción de celdas con frecuencia esperada menor a 5:  0.0
(0 de 4 celdas)

Nivel de significancia:  0.05
Resultado:  Las variables son independientes (No se rechaza la hipótesis nula)
0.29


In [ ]:
resultado = prueba_chi_cuadrada(df, 'tematica_analizada_2', 'sentimiento_comentario_2', alfa=0.05, imprimir=True)
print(resultado.round(2))